# 02_Modelado — MLY0100
**Tema:** Predicción y Evaluación de Riesgo de Diabetes  
**Autor:** Antonio Sepúlveda  
**Fecha:** 2025


# 1. Introducción

En este notebook realizamos el **modelado supervisado y análisis avanzado** aplicado al **riesgo de diabetes**, utilizando el dataset **preprocesado**.

Este notebook permite realizar:
- Entrenamiento manual del modelo (independiente del pipeline Kedro)
- Evaluación del modelo con métricas
- Matrices de confusión
- Comparación entre distintos clasificadores
- Visualizaciones clave del desempeño

Este análisis complementa el pipeline automatizado implementado en **Kedro**, funcionando como cuaderno exploratorio para justificar y validar los modelos.


# 2. Carga de datos preprocesados

Aquí cargamos el dataset previamente **limpiado** y guardado en:
`data/02_intermediate/diabetes_cleaned.csv`  

Este dataset ya está procesado según el notebook de EDA y el pipeline Kedro.


In [ ]:
# Cargar dataset limpio
import pandas as pd

df = pd.read_csv('../data/02_intermediate/diabetes_cleaned.csv')
df.head(), df.shape

# 3. Selección de variables

Definimos:
- **X** → variables de entrada
- **y** → variable objetivo (`Outcome`)

El dataset ya está normalizado salvo la columna objetivo, por lo que solo separamos las columnas.


In [ ]:
# Separar variables predictoras y objetivo
X = df.drop('Outcome', axis=1)
y = df['Outcome']
X.head()

# 4. División Train/Test

Se divide el dataset en:
- **80% entrenamiento**
- **20% prueba**

La misma división que usa Kedro, pero aplicada en este notebook.


In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)
len(X_train), len(X_test)

# 5. Modelado Supervisado

Entrenaremos varios clasificadores:

### Modelos incluidos:
- **Regresión Logística**
- **Random Forest**
- **Support Vector Machine (SVM)**
- **KNN**

Compararemos su desempeño para justificar cuál es el mejor candidato.


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report

modelos = {
    'Logistic Regression': LogisticRegression(max_iter=200),
    'Random Forest': RandomForestClassifier(n_estimators=200, random_state=42),
    'SVM (RBF)': SVC(kernel='rbf', probability=True),
    'KNN (k=5)': KNeighborsClassifier(n_neighbors=5)
}

resultados = {}

for nombre, modelo in modelos.items():
    modelo.fit(X_train, y_train)
    y_pred = modelo.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    resultados[nombre] = acc

resultados

## Ranking de modelos por Accuracy


In [ ]:
sorted(resultados.items(), key=lambda x: x[1], reverse=True)

# 6. Reporte con el mejor modelo

Obtenemos métricas detalladas del mejor clasificador encontrado.


In [ ]:
# Seleccionar mejor modelo
mejor_modelo_nombre = max(resultados, key=resultados.get)
mejor_modelo = modelos[mejor_modelo_nombre]

y_pred = mejor_modelo.predict(X_test)

print(f"🏆 Mejor Modelo: {mejor_modelo_nombre}")
print(classification_report(y_test, y_pred))

# 7. Matriz de Confusión

Una matriz de confusión permite visualizar el rendimiento del modelo por clases.

- TP → Verdaderos Positivos
- TN → Verdaderos Negativos
- FP → Falsos Positivos
- FN → Falsos Negativos


In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(5,4))
sns.heatmap(cm, annot=True, cmap='Blues', fmt='d')
plt.title('Matriz de Confusión — Mejor Modelo')
plt.xlabel('Predicción')
plt.ylabel('Real')
plt.show()

# 8. Curva ROC del mejor modelo

La curva ROC permite evaluar la capacidad del modelo para distinguir clases.
El AUC mide la calidad general del clasificador.


In [ ]:
from sklearn.metrics import roc_curve, auc

probs = mejor_modelo.predict_proba(X_test)[:,1]
fpr, tpr, _ = roc_curve(y_test, probs)
roc_auc = auc(fpr, tpr)

plt.figure(figsize=(6,5))
plt.plot(fpr, tpr, label=f'AUC = {roc_auc:.3f}')
plt.plot([0,1], [0,1], '--', color='gray')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Curva ROC — Mejor Modelo')
plt.legend()
plt.show()

# 9. Guardar el modelo (opcional)

Este archivo **no interfiere con el pipeline Kedro**, es solo para experimentar.
Pero puedes guardarlo con `joblib`.


In [ ]:
import joblib

joblib.dump(mejor_modelo, '../data/06_models/best_manual_model.pkl')
print('Modelo guardado en data/06_models/best_manual_model.pkl')